the classes will eventually be split up but they are put together for now to get their interactions correct <br>
sources: <br>
events: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-II/ <br>
portfolio: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-V/ <br>
execution handler: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-VI/

In [ ]:
import numpy as np
import pandas as pd
import os, sys

from dataclasses import dataclass

In [ ]:
quotes_filename = os.path.join("play_data", "XBTUSD_quotes_191214_0434.csv")
trades_filename = os.path.join("play_data", "XBTUSD_trades_191214_0434.csv")

In [ ]:
quotes_full = pd.read_csv(
    quotes_filename,
    index_col='recorded',
    parse_dates=['timestamp', 'recorded']
)

trades_full = pd.read_csv(
    trades_filename,
    index_col='received',
    parse_dates=['timestamp', 'received'],
    dtype={"side":"category"}
)

all_data = {
    "quotes": quotes_full,
    "trades": trades_full
}

display( quotes_full.head() )
display( trades_full.head() )

In [ ]:
@dataclass
class FillEvent:
    timestamp : pd.Timestamp
    symbol: str
    exchange: str
    quantity: int  # change in securities held
    fill_cost: float  # change in cash held
        
    def __post_init__(self):
        pass
    

        
class ExecutionHandler:
    
    def __init__(self, future_data):
        self.future_data = future_data  # df of bid/ask prices to calculate market fills with
        
    def send_order(self, order):
        pass
    
    def generate_fill(self, order):  # walks forward in time to see bid/ask prices
        pass

In [ ]:
eh = ExecutionHandler(quotes_full)
f = FillEvent(pd.Timestamp.utcnow(), "XBTUSD", "BitMEX", 10, 20)
f

In [ ]:
class Portfolio:
    def __init__(self, start_time, symbol_list, all_data, initial_capital=10000.0):
        
#         self.queue = queue  # event queue
        self.data = { k: df[:start_time] for k, df in all_data.items() }
        self.symbol_list = symbol_list 
        self.start_time = start_time
        self.initial_capital = initial_capital
        
        self.current_positions = pd.Series(0, index=symbol_list, name=start_time)
        self.current_holdings = self.construct_current_holdings(start_time)  # series
        
        self.all_positions_list = [self.current_positions]  # list of series
        self.all_holdings_list = [self.current_holdings]*10  # list of series
        
    @property
    def all_positions(self):
        return pd.DataFrame(self.all_positions_list)
    @property
    def all_holdings(self):
        return pd.DataFrame(self.all_holdings_list)
    
    def construct_current_holdings(self, time):  # market values of positions
        holdings = {symbol: 0.0 for symbol in symbol_list }
        holdings["capital"] = self.initial_capital
        holdings["commission"] = 0.0
        holdings["total"] = self.initial_capital
        return pd.Series(holdings, name=time)
    
    def update_fill(self, fill):
        pass
        
    def update_signal(self, signal):
        pass
    
symbol_list = "xbtusd", "ethusd"

p = Portfolio(pd.Timestamp.utcnow(), symbol_list, all_data)

p.current_holdings